# Final Project - Muntadhar AlZayer

Hello! This is my final project for *DTSA 5509 Introduction to Machine Learning: Supervised Learning*. I am using the data from the UCI Machine Learning Repository https://archive.ics.uci.edu/ 

I am using Bank Marketing (with social/economic context) Data Set that has the following description:
    *This dataset is using "in-vehicle coupon recommendation Data Set" UCI dataset.*
    
With the following 
            Attribute Information:

        destination: No Urgent Place, Home, Work
        passanger: Alone, Friend(s), Kid(s), Partner (who are the passengers in the car)
        weather: Sunny, Rainy, Snowy
        temperature:55, 80, 30
        time: 2PM, 10AM, 6PM, 7AM, 10PM
        coupon: Restaurant(<$20), Coffee House, Carry out & Take away, Bar, Restaurant($20-$50)
    expiration: 1d, 2h (the coupon expires in 1 day or in 2 hours)
    gender: Female, Male
    age: 21, 46, 26, 31, 41, 50plus, 36, below21
    maritalStatus: Unmarried partner, Single, Married partner, Divorced, Widowed
    has_Children:1, 0
    education: Some college - no degree, Bachelors degree, Associates degree, High School Graduate, Graduate degree (Masters or Doctorate), Some High School
    occupation: Unemployed, Architecture & Engineering, Student,
    Education&Training&Library, Healthcare Support,
    Healthcare Practitioners & Technical, Sales & Related, Management,
    Arts Design Entertainment Sports & Media, Computer & Mathematical,
    Life Physical Social Science, Personal Care & Service,
    Community & Social Services, Office & Administrative Support,
    Construction & Extraction, Legal, Retired,
    Installation Maintenance & Repair, Transportation & Material Moving,
    Business & Financial, Protective Service,
    Food Preparation & Serving Related, Production Occupations,
    Building & Grounds Cleaning & Maintenance, Farming Fishing & Forestry
    income: $37500 - $49999, $62500 - $74999, $12500 - $24999, $75000 - $87499,
    $50000 - $62499, $25000 - $37499, $100000 or More, $87500 - $99999, Less than $12500
    Bar: never, less1, 1~3, gt8, nan4~8 (feature meaning: how many times do you go to a bar every month?)
    CoffeeHouse: never, less1, 4~8, 1~3, gt8, nan (feature meaning: how many times do you go to a coffeehouse every month?)
    CarryAway:n4~8, 1~3, gt8, less1, never (feature meaning: how many times do you get take-away food every month?)
    RestaurantLessThan20: 4~8, 1~3, less1, gt8, never (feature meaning: how many times do you go to a restaurant with an average expense per person of less than $20 every month?)
    Restaurant20To50: 1~3, less1, never, gt8, 4~8, nan (feature meaning: how many times do you go to a restaurant with average expense per person of $20 - $50 every month?)
    toCoupon_GEQ15min:0,1 (feature meaning: driving distance to the restaurant/bar for using the coupon is greater than 15 minutes)
    toCoupon_GEQ25min:0, 1 (feature meaning: driving distance to the restaurant/bar for using the coupon is greater than 25 minutes)
    direction_same:0, 1 (feature meaning: whether the restaurant/bar is in the same direction as your current destination)
    direction_opp:1, 0 (feature meaning: whether the restaurant/bar is in the same direction as your current destination)
    Y:1, 0 (whether the coupon is accepted)
Source: https://archive.ics.uci.edu/ml/datasets/Bank+Marketing

Citation: [Moro et al., 2014] S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, Elsevier, 62:22-31, June 2014

# Project Summary

## Objective
I want to use this data to help predict the conditions and personal traits for people that take coupons when they are driving past. This will help the employees of those companies to better give out those coupons and to avoid conditions where people are less likely to accept them. The data includes both personal traits, weather conditions, and range of the establishment from the place that coupons are being given out. 

## Main model
The main model I will use is a Decision Tree Classifier as it utilizes all the features provided in the data, and will also help me to find the most important features. 

Note: While deciding when and where to give out coupons the employees will not be able to have personal trait details, but given the insight into the data we will be able to at least provide them with insight into external factors. 

## Import Libraries and Data
We start with importing necessary libraries:

In [ ]:
%matplotlib inline
!pip uninstall scikit-learn
!pip install  scikit-learn==0.23.2
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np
import scipy as sp
import scipy.stats as stats
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Set color map to have light blue background
sns.set()
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn import tree


Source: https://archive.ics.uci.edu/ml/datasets/Bank+Marketing

### Data Import

We then import the data from the data file:

In [ ]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00603/in-vehicle-coupon-recommendation.csv')

Below we see a sample of the data

In [ ]:
df.head()

There doesn't seem to be any NULL values but they are set to unknown

In [ ]:
print(df.info())
df.describe()

We initially have 25 features with 12684 rows, as we go through data cleaning this may change. Please find the description for each feature above.

### Data Cleaning

We want to find the columns that have null values and the course of action for each.

In [ ]:
for c in df.columns:
    if(df[c].isnull().sum() / len(df) >0 and df[c].isnull().sum() / len(df) <= .05):
        print("Impute column '",c, "'  - Percenteage of null values: ", df[c].isnull().sum() / len(df))
    elif (df[c].isnull().sum() / len(df)) > .05:
        print("Drop column '", c, "'  - Percenteage of null values: ", df[c].isnull().sum() / len(df))

The column `car` has 99.14% missing values so it needs to be dropped as it is above the 5% acceptable null values. The rest of the columns that contain null values can be imputed with the mode of each as they are all categorical columns.  

In [ ]:
# Drop column 'car'
df = df.drop(columns=['car'])

In [ ]:
#Impute the remaining columns with null values
cols = df.columns
feats_w_null = []
for c in df.columns:
    if df[c].isnull().sum() > 0:
        feats_w_null.append(c)


print(feats_w_null)


In [ ]:
#For each column in feats_w_null we get the mode and replace all null values with the most common entry

for x in feats_w_null:
    replacement = df[x].mode()
    df[x].fillna(replacement[0], inplace=True)
    print("Filled nulls for", x)


In [ ]:
for c in df.columns:
    if(df[c].isnull().sum() / len(df) >0 and df[c].isnull().sum() / len(df) <= .05):
        print("Impute: ",c)
    elif (df[c].isnull().sum() / len(df)) > .05:
        print("Drop :", c)

In [ ]:
print(df.info())

After data cleaning, we now have no null values and 24 features with the same number of rows. 

### Data Visualization
Lets take a deeper dive into the categorical data and visualize them

#### Destination

In [ ]:
df['destination'].value_counts().plot(kind='barh')

In [ ]:
destination = df.groupby(['destination', 'Y']).size().unstack()
destination

From this we can see that there is a split between "No Urgent Place" and the other two categories of "Work" and "Home". This is a good indicator that the coupons are being spread at different times of the day and during different days of the week. As well, we see almost a 50/50 split within the outcome based on the value of the persons destination.

#### Weather

In [ ]:
df['weather'].value_counts().plot(kind='barh')

In [ ]:
weather = df.groupby(['weather', 'Y']).size().unstack()
weather

There is about a 50/50 split for people accepting the coupon based on the weather, so we can see that there is no direct correlation between the two visalized features above.

#### toCoupon_GEQ5min

In [ ]:
df['toCoupon_GEQ5min'].value_counts().plot(kind='barh')

The values of `toCoupon_GEQ5min` are all 1 so we can drop this column as it does not provide valuable data. 

In [ ]:
df = df.drop(columns=['toCoupon_GEQ5min'])

Now we have 23 features to work with.

# Categorical string to int
We need to change all string categorical data into integer. I utilized the LabelEncoder function in sklearn to do this instead of manually mapping each instance to a value.

In [ ]:
dtypes = df.dtypes #Data Types for each column
columns = df.columns # Columns in dataframe
for x in columns:
    if dtypes[x] == object: #If the values are not continous
        print("Column:",x, '\n', " String categories:",df[x].unique()) #Print old string categories
        le = preprocessing.LabelEncoder() #Init LabelEncoder
        le.fit(df[x].unique()) #Fit it with the unique values in the dataframe column
        df[x] = le.transform(df[x]) #Transform them into their integer values
        print( "  Int categories:",df[x].unique()) #Print new values

In [ ]:
print(df.info())

Now we have all our columns in numerical form with both continous and categorical data. 

# Data Correlation
Lets test the correlation between the data and find the most correlated feature to our result

In [ ]:
sns.heatmap(df.corr()) #Heatmap for correlations in dataframe

In [ ]:
#Y is the last column/row in the correlation above, we can quantify it below:
correlation = df.corr()['Y']
print(correlation.sort_values(ascending=False))

Even the most correlated feature is not correlated enough to utilize a Linear model, but lets test it with some of the features:

In [ ]:
model = smf.ols(formula='Y ~ weather + coupon + CoffeeHouse ', data=df)
 
res = model.fit() #update this value according to the result
print(res.summary())

Above is a Linear Model for testing the highest correlated features, even then we don't have enough to predict future values based solely on `weather`, `coupon`, and `expiration`. So we need to use Decision Tree Classifier to have the best use of all features.

# Decision Tree Classifier

First, we move the features and the result into x and y respectively

In [ ]:
#Split data into parameters and result 
x = df.drop(columns='Y').copy()
y = df[['Y']].copy()

print(x.info())
# print(y.info())

Then we provide a 80/20 split for trainging and testing data

In [ ]:
#Split data into training and testing 
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8)


In [ ]:
x_train

With the data ready, we can then run random parameters on the DTC and view the accuracy score

In [ ]:
#Decision Tree Classifier with random initial parameters 
classifier = DecisionTreeClassifier(max_depth=10, random_state=14)
classifier.fit(x_train, y_train)
pred = classifier.predict(x_test)
acc_score = accuracy_score(y_true=y_test, y_pred = pred)

print("Accuracy Score for initial DTC:", acc_score)

An accuracy score of 69% is nice for random initial values but we can get a better score by modifying our parameters, we can run GridSearchCV to search for the best values to give us the best Accuracy of the model

In [ ]:
parameters = {'max_depth' : np.arange(3,10),
             'criterion': ['gini', 'entropy'],
              'max_leaf_nodes' : [5,10,15,20,50,100],
              'min_samples_split' : [2,4,5,10,15,20]
             }
grid_search_tree = GridSearchCV(DecisionTreeClassifier(), parameters, scoring="accuracy")

grid_search_tree.fit(x_train, y_train)

print("Best Estimator values:", grid_search_tree.best_estimator_)

print('Best Score:', np.abs(grid_search_tree.best_score_))

With the best parameters provided to us above, we can then use that model to predict the test data and get the accuracy score

In [ ]:
#Testing the best scoring parameters from above with test value:

classifier = DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=9, max_features=None, max_leaf_nodes=100,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=15,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

classifier.fit(x_train, y_train)
pred = classifier.predict(x_test)
acc_score = accuracy_score(y_true=y_test, y_pred = pred)
f_score = f1_score(y_true=y_test, y_pred=pred)
print("Accuracy Score Optimized Parameters:", acc_score)
print("F1 Score Optimized Parameters:", f_score)


With the test data we get around the same accuracy score, now lets look at the most important features below:

In [ ]:
#Feature importance
features = x.columns
scores = classifier.feature_importances_.tolist()
res = pd.DataFrame({'features' : features, 'score': scores})
res = res.sort_values(by=['score'], ascending=False)
print(res)

We can thus conclude that the most important identifier from the Decision Tree Classifier model is the type of coupon that is presented to the customer, the frequency the person accepting the coupon goes to a Coffe House within a month, and if the driving time to the establishment is >=25 minutes.

We can see how our models accuracy will be if we choose the top 1, 2, 3 features provided above. 

In [ ]:
#models with top 1, 2, & 3 features
top_1 = df[['coupon']].copy()
top_2 = df[['coupon', 'Bar']].copy()
top_3 = df[['coupon', 'Bar', 'income']].copy()

x_train_1, x_test_1, y_train_1, y_test_1 = train_test_split(top_1,y, test_size=.2)
x_train_2, x_test_2, y_train_2, y_test_2 = train_test_split(top_2,y, test_size=.2)
x_train_3, x_test_3, y_train_3, y_test_3 = train_test_split(top_3,y, test_size=.2)

classifier.fit(x_train_1, y_train_1)
pred1 = classifier.predict(x_test_1)
acc_score1 = accuracy_score(y_true=y_test_1, y_pred = pred1)
print("With the top (1) feature, accuracy score = ", acc_score1)

classifier.fit(x_train_2, y_train_2)
pred2 = classifier.predict(x_test_2)
acc_score2 = accuracy_score(y_true=y_test_2, y_pred = pred2)
print("With the top (2) features, accuracy score = ", acc_score2)

classifier.fit(x_train_3, y_train_3)
pred3 = classifier.predict(x_test_3)
acc_score3 = accuracy_score(y_true=y_test_3, y_pred = pred3)
print("With the top (3) features, accuracy score = ", acc_score3)

Accuracy fluctuates with the addition of additional parameters. But, there is an increase in accuracy as we add more features.

## Confusion Matrix 
For the optimized DTC model we created above, we want to view how we can improve the values of FN and FP. First lets view the amounts for those:

In [ ]:
#Confusion Matrix
cm = sklearn.metrics.confusion_matrix(y_test, pred)
print(cm)

TP = 0
FP = 0
TN = 0
FN = 0

y_true = y_test.values.tolist()
pos_label_value = 1
for l in range(len(pred)):
    predicted = pred[l]
    true = y_true[l][0]
    if predicted == pos_label_value and true == pos_label_value:
        TP += 1
    elif predicted == pos_label_value and true != pos_label_value:
        FP += 1
    elif predicted != pos_label_value and true == pos_label_value:
        FN += 1
    elif predicted != pos_label_value and true != pos_label_value:
        TN += 1
print("TP = ", TP)
print("FP = ", FP)
print("TN = ", TN)
print("FN = ", FN)

To improve those amounts we can change the GridSearchCV scoring to f1 instaed of accuracy to the predictions. This should help us to predict the TP and TN values a bit better. 

In [ ]:
parameters = {'max_depth' : np.arange(3,10),
             'criterion': ['gini', 'entropy'],
              'max_leaf_nodes' : [5,10,15,20,50,100],
              'min_samples_split' : [2,4,5,10,15,20]
             }
grid_search_tree = GridSearchCV(DecisionTreeClassifier(), parameters, scoring="f1")

grid_search_tree.fit(x_train, y_train)

print("Best Estimator values:", grid_search_tree.best_estimator_)

print('Best Score:', np.abs(grid_search_tree.best_score_))

In [ ]:
classifier = DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=7, max_features=None, max_leaf_nodes=50,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

classifier.fit(x_train, y_train)
pred = classifier.predict(x_test)
acc_score = accuracy_score(y_true=y_test, y_pred = pred)
f_score = f1_score(y_true=y_test, y_pred=pred)
print("Accuracy Score Optimized Parameters:", acc_score)
print("F1 Score Optimized Parameters:", f_score)

In [ ]:
features = x_train.columns
scores = classifier.feature_importances_.tolist()
res = pd.DataFrame({'features' : features, 'score': scores})
res = res.sort_values(by=['score'], ascending=False)
print(res)

In [ ]:
#Confusion Matrix
cm = sklearn.metrics.confusion_matrix(y_test, pred)
print(cm)

TP_new = 0
FP_new = 0
TN_new = 0
FN_new = 0

y_true = y_test.values.tolist()
pos_label_value = 1
for x in range(len(pred)):
    predicted = pred[x]
    true = y_true[x][0]
    if predicted == pos_label_value and true == pos_label_value:
        TP_new += 1
    elif predicted == pos_label_value and true != pos_label_value:
        FP_new += 1
    elif predicted != pos_label_value and true == pos_label_value:
        FN_new += 1
    elif predicted != pos_label_value and true != pos_label_value:
        TN_new += 1
print("New TP = ", TP_new)
print("New FP = ", FP_new)
print("New TN = ", TN_new)
print("New FN = ", FN_new)


To compare those values we can see the following:

TP =  1136, New TP =  1159

FP =  423, New FP =  467

TN =  649, New TN =  605

FN =  329, New FN =  306

Both False Negatives and true Negatives decreased, while False Positives and True Positives increased. This may be a favorable for this model and dataset as we prefer to include more people into positive to provide them with the ability to accept the coupon despite them being a possible negative as they may be persuaded by further external factors not included in this dataset. 

### Feature Engineering
For this I want to remove all personal traits, and focus on observable features. 

I will start with combining direction_opp and direction_same into one column with different values:

In [ ]:
fe_df = df.copy()
fe_df['direction'] = 0
fe_df.loc[((fe_df.direction_opp == 1) & (fe_df.direction_same == 0)), 'direction'] = 0 # Opposite direction
fe_df.loc[((fe_df.direction_opp == 0) & (fe_df.direction_same == 1)), 'direction'] = 1 # Same direction
fe_df = fe_df.drop(columns=['direction_same', 'direction_opp'])

Next we will make the passenger column to be a binary, to make it more of an observable feature:

In [ ]:
fe_df['b_passanger'] = 0
fe_df.loc[((fe_df['passanger'] == 0), 'b_passanger')] = 0
fe_df.loc[((fe_df['passanger'] == 1) | (fe_df['passanger'] == 2) | (fe_df['passanger'] == 3), 'b_passanger')] = 1
fe_df = fe_df.drop(columns=['passanger'])
fe_df = fe_df.rename(columns={'b_passanger' : 'new_passanger'})

Finally, we want to combine the columns `toCoupon_GEQ15min` and `toCoupon_GEQ25min`:

In [ ]:
fe_df['toCoupon'] = 0
fe_df.loc[((fe_df['toCoupon_GEQ15min'] == 0) & (fe_df['toCoupon_GEQ25min'] == 0), 'toCoupon')] = 0 # Less than 15 mins
fe_df.loc[((fe_df['toCoupon_GEQ15min'] == 1) & (fe_df['toCoupon_GEQ25min'] == 0), 'toCoupon')] = 1 # between 15 and 25
fe_df.loc[((fe_df['toCoupon_GEQ15min'] == 1) & (fe_df['toCoupon_GEQ25min'] == 1), 'toCoupon')] = 2 # Greater than 25
fe_df = fe_df.drop(columns=['toCoupon_GEQ15min', 'toCoupon_GEQ25min'])

In [ ]:
fe_df = fe_df[['destination', 'new_passanger', 'weather', 'temperature', 'direction', 'time', 'coupon', 'expiration', 'toCoupon', 'Y']].copy()

In [ ]:
fe_df.head()

## Feature Engineered visualization

Lets take a look into the combined columns:

In [ ]:
fe_df['direction'].value_counts().plot(kind='barh')

In [ ]:
direction = fe_df.groupby(['direction', 'Y']).size().unstack()
direction

In [ ]:
fe_df['new_passanger'].value_counts().plot(kind='barh')

In [ ]:
passanger = fe_df.groupby(['new_passanger', 'Y']).size().unstack()
passanger

### Re-run model on new dataset

In [ ]:
fe_x = fe_df.drop(columns='Y').copy()
fe_y = fe_df[['Y']].copy()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(fe_x, fe_y, train_size=0.8)

In [ ]:
parameters = {'max_depth' : np.arange(3,10),
             'criterion': ['gini', 'entropy'],
              'max_leaf_nodes' : [5,10,15,20,50,100],
              'min_samples_split' : [2,4,5,10,15,20]
             }
grid_search_tree = GridSearchCV(DecisionTreeClassifier(), parameters, scoring="accuracy")

grid_search_tree.fit(x_train, y_train)

print("Best Estimator values:", grid_search_tree.best_estimator_)

print('Best Score:', np.abs(grid_search_tree.best_score_))

In [ ]:
classifier = DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=9, max_features=None, max_leaf_nodes=100,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

classifier.fit(x_train, y_train)
pred = classifier.predict(x_test)
acc_score = accuracy_score(y_true=y_test, y_pred = pred)
f_score = f1_score(y_true=y_test, y_pred=pred)
print("Accuracy Score Optimized Parameters:", acc_score)
print("F1 Score Optimized Parameters:", f_score)

In [ ]:
features = x_train.columns
scores = classifier.feature_importances_.tolist()
res = pd.DataFrame({'features' : features, 'score': scores})
res = res.sort_values(by=['score'], ascending=False)
print("Feature importance")
print(res)

In [ ]:
cm = sklearn.metrics.confusion_matrix(y_test, pred)
print(cm)

fe_TP = 0
fe_FP = 0
fe_TN = 0
fe_FN = 0

y_true = y_test.values.tolist()
pos_label_value = 1
for x in range(len(pred)):
    predicted = pred[x]
    true = y_true[x][0]
    if predicted == pos_label_value and true == pos_label_value:
        fe_TP += 1
    elif predicted == pos_label_value and true != pos_label_value:
        fe_FP += 1
    elif predicted != pos_label_value and true == pos_label_value:
        fe_FN += 1
    elif predicted != pos_label_value and true != pos_label_value:
        fe_TN += 1
print("With max accuracy we get:")
print("FE Acc TP = ", fe_TP)
print("FE Acc FP = ", fe_FP)
print("FE Acc TN = ", fe_TN)
print("FE Acc FN = ", fe_FN)

In [ ]:
parameters = {'max_depth' : np.arange(3,10),
             'criterion': ['gini', 'entropy'],
              'max_leaf_nodes' : [5,10,15,20,50,100],
              'min_samples_split' : [2,4,5,10,15,20]
             }
grid_search_tree = GridSearchCV(DecisionTreeClassifier(), parameters, scoring="f1")

grid_search_tree.fit(x_train, y_train)

print("Best Estimator values:", grid_search_tree.best_estimator_)

print('Best Score:', np.abs(grid_search_tree.best_score_))

In [ ]:
classifier = DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=6, max_features=None, max_leaf_nodes=50,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

classifier.fit(x_train, y_train)
pred = classifier.predict(x_test)
acc_score = accuracy_score(y_true=y_test, y_pred = pred)
f_score = f1_score(y_true=y_test, y_pred=pred)
print("Accuracy Score Optimized Parameters:", acc_score)
print("F1 Score Optimized Parameters:", f_score)

In [ ]:
features = x_train.columns
scores = classifier.feature_importances_.tolist()
res = pd.DataFrame({'features' : features, 'score': scores})
res = res.sort_values(by=['score'], ascending=False)
print("Feature importance")
print(res)

In [ ]:
cm = sklearn.metrics.confusion_matrix(y_test, pred)
print(cm)

fe_TP = 0
fe_FP = 0
fe_TN = 0
fe_FN = 0

y_true = y_test.values.tolist()
pos_label_value = 1
for x in range(len(pred)):
    predicted = pred[x]
    true = y_true[x][0]
    if predicted == pos_label_value and true == pos_label_value:
        fe_TP += 1
    elif predicted == pos_label_value and true != pos_label_value:
        fe_FP += 1
    elif predicted != pos_label_value and true == pos_label_value:
        fe_FN += 1
    elif predicted != pos_label_value and true != pos_label_value:
        fe_TN += 1
print("With max f1 we get:")
print("FE f1 TP = ", fe_TP)
print("FE f1 FP = ", fe_FP)
print("FE f1 TN = ", fe_TN)
print("FE f1 FN = ", fe_FN)

Now that we only ran our model on observable features, we can see that the model improved, and will help the people passing out the coupons to use their observation skills, and decision for time and direction to pass out the coupons more effectively to people that are more likely to 

# Conclusions

The model before feature engineering may not be the best even after the changes made to be more focused on f1-score rather than accuracy. But, having more positives than negatives in this situation is not the worse as it would lead to more wasted time of offering coupons to those that may not accept them, but it also will lead to more true positive outcomes. 

After choosing only the obersable features we got a worse model, but unless we have a camera that views into the car and gives us the backstory and details for each person it will not be a practical model. This way we can provide those employees with the insight into what factors that they can observe to offer a coupon to a driver that is more likely to accept a coupon and improve their effeciency. 

To improve this we might need to choose a different model that uses weights on each feature to have better insight into which feature will impact the customer not accepting more. I would recommend a bayesian model as well. 